In [12]:
import numpy as np
import sklearn
import pandas as pd
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

In [6]:
# with open() as fp:
#     game_players= pickle.load(fp)
with open('player_data/train_processed_player_data.pkl', 'rb') as fp:
    train_processed_player_data = pickle.load(fp)
with open('player_cluster_scaler.pkl', 'rb') as fp:
    player_scaler = pickle.load(fp)
with open('player_cluster.pkl', 'rb') as fp:
    player_cluster = pickle.load(fp)
    

In [7]:
def gen_feature(season, processed_player_data, player_scaler, player_cluster, box_info, starter_weight, num_clust):
    t1_vec = np.zeros((1, num_clust * 2))
    t2_vec = np.zeros((1, num_clust * 2))
    t1_players, t2_players, scores = box_info
    t1_starters = t1_players[:5]
    t1_bench = t1_players[5:]
    
    t2_starters = t2_players[:5]
    t2_bench = t2_players[5:]
    
    for player in t1_starters:
        player_stats = processed_player_data[player][season]
        scaled_stat = player_scaler.transform([player_stats])
        clust_num = player_cluster.predict(scaled_stat)[0]
        t1_vec[clust_num] += starter_weight
        t2_vec[clust_num + num_clust] += starter_weight
    for player in t1_bench:
        player_stats = processed_player_data[player][season]
        scaled_stat = player_scaler.transform([player_stats])
        clust_num = player_cluster.predict(scaled_stat)[0]
        t1_vec[clust_num] += 1 - starter_weight
        t2_vec[clust_num + num_clust] += 1 - starter_weight
    for player in t2_starters:
        player_stats = processed_player_data[player][season]
        scaled_stat = player_scaler.transform([player_stats])
        clust_num = player_cluster.predict(scaled_stat)[0]
        t2_vec[clust_num] += starter_weight
        t1_vec[clust_num + num_clust] += starter_weight
        
    
    for player in t2_bench:
        player_stats = processed_player_data[player][season]
        scaled_stat = player_scaler.transform([player_stats])
        clust_num = player_cluster.predict(scaled_stat)[0]
        t2_vec[clust_num] += 1 - starter_weight
        t1_vec[clust_num + num_clust] += 1 - starter_weight
        
        
    return ((t1_vec, scores[0]), (t2_vec, score[1]))


      
        
        
        
    
    
    
    

{'carmelo-anthony': {'2012-2013': [37.0,
   22.2,
   0.449,
   6.2,
   0.379,
   7.6,
   0.83,
   6.9,
   2.6,
   0.5,
   0.8,
   3.1,
   2.6,
   28.7],
  '2013-2014': [38.7,
   21.3,
   0.452,
   5.4,
   0.402,
   7.0,
   0.848,
   8.1,
   3.1,
   0.7,
   1.2,
   2.9,
   2.6,
   27.4],
  '2014-2015': [35.7,
   20.2,
   0.444,
   4.5,
   0.341,
   5.9,
   0.797,
   6.6,
   3.1,
   0.4,
   1.0,
   2.2,
   2.2,
   24.2],
  '2015-2016': [35.1,
   18.2,
   0.434,
   4.3,
   0.339,
   5.6,
   0.829,
   7.7,
   4.2,
   0.5,
   0.9,
   2.5,
   2.4,
   21.8],
  '2016-2017': [34.3,
   18.8,
   0.433,
   5.7,
   0.359,
   4.9,
   0.833,
   5.9,
   2.9,
   0.4,
   0.8,
   2.7,
   2.1,
   22.4]},
 'kevin-durant': {'2012-2013': [38.5,
   17.7,
   0.51,
   4.1,
   0.416,
   9.3,
   0.905,
   7.9,
   4.6,
   1.3,
   1.4,
   1.8,
   3.5,
   28.1],
  '2013-2014': [38.5,
   20.8,
   0.503,
   6.1,
   0.391,
   9.9,
   0.873,
   7.4,
   5.5,
   0.7,
   1.3,
   2.1,
   3.5,
   32.0],
  '2014-2015': [33.8,

In [ ]:
train_season = ['2012-2013', '2013-2014', '2014-2015', '2015-2016']
test_season = ['2016-2017']

# train_X = np.zeros((len(train_season) * 1230 * 2, num_clust * 2))
# train_y = np.zeros((len(train_season) * 1230 * 2,))

# test_X = np.zeros((len(test_season) * 1230 * 2, num_clust * 2))
# test_y = np.zeros((len(test_season) * 1230 * 2,))

train_X =[]
train_y = []
test_X = []
test_y = []

for year in train_season:
    for box_info in game_statinfo[year]:
        t1_info, t2_info = gen_feature(year, train_processed_player_data, player_scaler, player_cluster, box_info, starter_weight, num_clust)
        t1_vec, t1_score = t1_info
        t2_vec, t2_score = t2_info
        
        train_X.append(t1_vec)
        train_y.append(t1_score)
        train_X.append(t2_vec)
        train_y.append(t2_score)
        
        
train_X  = np.array(train_X)
train_Y = np.array(train_Y)

for year in test_season:
    for box_info in game_statinfo[year]:
        t1_info, t2_info = gen_feature(year, test_processed_player_data, player_scaler, player_cluster, box_info, starter_weight, num_clust)
        t1_vec, t1_score = t1_info
        t2_vec, t2_score = t2_info
        
        test_X.append(t1_vec)
        test_y.append(t1_score)
        test_X.append(t2_vec)
        test_y.append(t2_score)
        
        
test_X  = np.array(test_X)
test_Y = np.array(test_Y)



        
        
        
    
    
    
    

In [10]:

#Model1 Linear Regression

reg = LinearRegression().fit(train_X, train_Y)
y_pred = reg.predict(test_X)
mse = mean_squared_error(y_pred, test_Y)

correct_count = 0
total_count = 0
for year in test_season:
    for box_info in game_statinfo[year]:
        t1_info, t2_info = gen_feature(year, test_processed_player_data, player_scaler, player_cluster, box_info, starter_weight, num_clust)
        t1_vec, t1_score = t1_info
        t2_vec, t2_score = t2_info
        
        t1_preds = reg.predict([t1_vec])
        t2_preds = reg.predict([t2_vec])
        pred_res = t1_preds > t2_preds
        true_res = t1_score > t2_score
        if pred_res == true_res:
            correct_count += 1
        total_count += 1
        
print(correct_count / total_count)

        



In [ ]:
#Model2 SVM Regression

reg = SVR().fit(train_X, train_Y)
y_pred = reg.predict(test_X)
mse = mean_squared_error(y_pred, test_Y)

correct_count = 0
total_count = 0
for year in test_season:
    for box_info in game_statinfo[year]:
        t1_info, t2_info = gen_feature(year, test_processed_player_data, player_scaler, player_cluster, box_info, starter_weight, num_clust)
        t1_vec, t1_score = t1_info
        t2_vec, t2_score = t2_info
        
        t1_preds = reg.predict([t1_vec])
        t2_preds = reg.predict([t2_vec])
        pred_res = t1_preds > t2_preds
        true_res = t1_score > t2_score
        if pred_res == true_res:
            correct_count += 1
        total_count += 1
        
print(correct_count / total_count)